In [1]:
# creating empty list for output
topiclink = []
title_list = []
profile = []
date = []
abstract = []
names = []


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


def get_text(item_url):
    """_summary_

    Args:
        item_url (_type_): _description_
    """
    sourceCode = requests.get(item_url)
    plainText = sourceCode.text
    soup = BeautifulSoup(plainText, "html.parser")
    for i in soup.find_all("div", class_="textblock"):
        try:
            abs = i.getText()
        except ValueError:
            abs = None
        abstract.append(abs)

    for person in soup.find_all("a", class_="link person"):
        try:
            person_link = person.get("href")
        except ValueError:
            person_link = None
        profile.append(person_link)

        try:
            name = person.span.text.strip()
        except ValueError:
            name = None
        names.append(name)

# crawling the details from coventry university


def create_cov(max_pages):

    page = 0

    while(page <= max_pages):
        URL = ("https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=" + str(page))
        page += 1
        print("fetching " + URL)
        source_code = requests.get(URL)
        plainText = source_code.text
        soup = BeautifulSoup(plainText, "html.parser")
        for link in soup.find_all("div", {"class": ["result-container", "link person"]}):
            link_url = link.a.get('href')
            if(link_url != None and link_url != "/"):
                link_url = link_url.strip()
            else:
                link_url = None
                print("new url is : ", link_url)
            topiclink.append(link_url)
            try:
                time = link.find("span", class_="date")
                time = time.text.strip()
            except ValueError:
                time = None
            date.append(time)
            try:
                title = link.a.text.strip()
            except ValueError:
                title = None
                print("empty")
            title_list.append(title)

            get_text(link_url)

    else:
        print('end')


create_cov(12)


fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=0
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=1
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=2
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=3
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=4
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=5
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=6
fetching https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=7
fetching

In [5]:
zipped = list(zip(title_list, topiclink, abstract, names, profile, date))
uni_df = pd.DataFrame(zipped, columns=[
                      "research title", "publication link", "abstract", "author name", "pureportal link", "date"])
uni_df["ID"] = [x for x in range(1, len(uni_df.values) + 1)]


In [6]:
uni_df.head(5)


,research title,publication link,abstract,author name,pureportal link,date,ID
0,A bibliometric review of the Waqf literature,https://pureportal.coventry.ac.uk/en/publicati...,Waqf received research attention due to its ab...,Rashedul Hasan,https://pureportal.coventry.ac.uk/en/persons/r...,Jun 2022,1
1,A note on COVID-19 instigated maximum drawdown...,https://pureportal.coventry.ac.uk/en/publicati...,This study uncovers the impact of the COVID-19...,Rashedul Hasan,https://pureportal.coventry.ac.uk/en/persons/r...,May 2022,2
2,Bank stock valuation theories: do they explain...,https://pureportal.coventry.ac.uk/en/publicati...,PurposeThe objective of this paper is to inves...,Alireza Zarei,https://pureportal.coventry.ac.uk/en/persons/a...,1 Mar 2022,3
3,CEO Duality and Firm Performance: A Systematic...,https://pureportal.coventry.ac.uk/en/publicati...,This paper systematically reviews 314 empirica...,Mei Yu,https://pureportal.coventry.ac.uk/en/persons/m...,25 May 2022,4
4,CEO Financial Experience and Firms’ Earnings M...,https://pureportal.coventry.ac.uk/en/publicati...,This study investigates the impact of CEOs’ fi...,Thai Nguyen,https://pureportal.coventry.ac.uk/en/persons/t...,7 Mar 2022,5


In [7]:
uni_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654 entries, 0 to 653
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   research title    654 non-null    object
 1   publication link  654 non-null    object
 2   abstract          654 non-null    object
 3   author name       654 non-null    object
 4   pureportal link   654 non-null    object
 5   date              654 non-null    object
 6   ID                654 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 35.9+ KB


In [8]:
coventry = uni_df.copy()
coventry = coventry.astype(str)
entry = coventry.loc[1, :].copy()
print(entry)

research title      A note on COVID-19 instigated maximum drawdown...
publication link    https://pureportal.coventry.ac.uk/en/publicati...
abstract            This study uncovers the impact of the COVID-19...
author name                                            Rashedul Hasan
pureportal link     https://pureportal.coventry.ac.uk/en/persons/r...
date                                                         May 2022
ID                                                                  2
Name: 1, dtype: object


In [61]:
# this function is created to convert all the text to lower text and to remove punctuation to avoid it to not be case sensitve when searching
def process_string(text):
    text = text.lower()  # to lowercase
    text = text.translate(
        str.maketrans("", "", string.punctuation)
    )  # strip punctuation
    return text

process_string(entry.abstract)

'this study uncovers the impact of the covid19 on the islamic equity markets compared to their conventional counterparts the extremely largescale drawdown across the markets signifies an indiscriminate impact to some extent asian islamic markets show relative resilience to their counterparts both islamic and nonislamic asian markets signpost a quicker recovery than the rest of the regions the middle east  africa europe and america it appears that a higher return leads to a smaller maximum drawdown while higher volatility leads to a larger maximum drawdown despite the largescale drawdown a number of markets secure a positive return where islamic markets outperform the counterparts conventional markets respond to the covid19 aftershock homogenously as a result of their high interlinkages collectively these results reinforce the view that in the crisis period islamic markets are more resilient'

In [59]:
# load nltk's English stopwords as variable called 'stopwords'
import nltk
nltk.download("omw-1.4")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
import string



# function to tag first charater lemmatize accepts
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV,
    }

    return tag_dict.get(tag, wordnet.NOUN)

stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[:10])



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ajaoo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ajaoo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajaoo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajaoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ajaoo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
# Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item
lem = WordNetLemmatizer()

# this function loops through the words by properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word
def stop_lemmatize(doc):
    tokens = word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stopwords:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp


stop_lemmatize(doc=entry.abstract)
stop_lemmatize(doc=entry["research title"])
coventry_processed = coventry.copy()

In [64]:
# a function to transform a columns in the dataframe and return the end resut which has been processed
def transform_df(df):
    df = df
    df["Name"] = df["author name"].apply(process_string)
    df["topic"] = df["research title"].apply(process_string)
    df["abstract"] = df["abstract"].apply(process_string)
    df["text"] = df["Name"] + " " + df["topic"] + " " + df["abstract"]
    drop_cols = [
        "Name",
        "publication link",
        "pureportal link",
        "abstract",
        'date',
        "topic",
        "author name",
        "research title",
        "topic"
    ]
    df = df.drop(drop_cols, axis=1)
    return df

merged_trans = transform_df(coventry_processed)
merged_trans

# 0   research title    654 non-null    object
#  1   publication link  654 non-null    object
#  2   abstract          654 non-null    object
#  3   author name       654 non-null    object
#  4   pureportal link   654 non-null    object
#  5   date              654 non-null    object
#  6   ID                654 non-null    int64

,ID,text
0,1,rashedul hasan a bibliometric review of the wa...
1,2,rashedul hasan a note on covid19 instigated ma...
2,3,alireza zarei bank stock valuation theories do...
3,4,mei yu ceo duality and firm performance a syst...
4,5,thai nguyen ceo financial experience and firms...
...,...,...
649,650,eliana lauretta testing the weakform efficienc...
650,651,eliana lauretta the consequences of a refund i...
651,652,mei yu the corporate social responsibility of ...
652,653,mehtap hisarciklilar the credit signals that m...


In [65]:
entry = coventry_processed.loc[3,:].copy()
print(entry)
ID = entry.ID
index_test = {}

research title      CEO Duality and Firm Performance: A Systematic...
publication link    https://pureportal.coventry.ac.uk/en/publicati...
abstract            this paper systematically reviews 314 empirica...
author name                                                    Mei Yu
pureportal link     https://pureportal.coventry.ac.uk/en/persons/m...
date                                                      25 May 2022
ID                                                                  4
Name                                                           mei yu
topic               ceo duality and firm performance a systematic ...
text                mei yu ceo duality and firm performance a syst...
Name: 3, dtype: object


In [66]:
words = entry.text.split(" ")
word = words[0]
sample = {word: [ID]}
print(sample)

{'mei': ['4']}


In [68]:
for word in words:
  if word in index_test.keys():
    if ID not in index_test[word]:
      index_test[word].append(ID)
  else:
    index_test[word] = [ID]
    
print(index_test)

{'mei': ['4'], 'yu': ['4'], 'ceo': ['4'], 'duality': ['4'], 'and': ['4'], 'firm': ['4'], 'performance': ['4'], 'a': ['4'], 'systematic': ['4'], 'review': ['4'], 'research': ['4'], 'agenda': ['4'], 'this': ['4'], 'paper': ['4'], 'systematically': ['4'], 'reviews': ['4'], '314': ['4'], 'empirical': ['4'], 'studies': ['4'], 'which': ['4'], 'examine': ['4'], 'the': ['4'], 'relationship': ['4'], 'between': ['4'], 'board': ['4'], 'leadership': ['4'], 'structure': ['4'], 'results': ['4'], 'show': ['4'], 'that': ['4'], 'mixed': ['4'], 'findings': ['4'], 'are': ['4'], 'due': ['4'], 'to': ['4'], 'different': ['4'], 'measurements': ['4'], 'designs': ['4'], 'sampling': ['4'], 'practices': ['4'], 'approaches': ['4'], 'of': ['4'], 'dealing': ['4'], 'with': ['4'], 'endogeneity': ['4'], 'issues': ['4'], 'utilizing': ['4'], 'multiplecountry': ['4'], 'data': ['4'], 'multinational': ['4'], 'companies': ['4'], 'small': ['4'], 'firms': ['4'], 'or': ['4'], 'regions': ['4'], 'covering': ['4'], 'africa': ['4'

In [74]:
def index_it(entry, index):
  words = entry.text.split()
  ID = entry.ID
  for word in words:
    if word in index.keys():
      if ID not in index[word]:
        index[word].append(ID)
    else:
      index[word] = [ID]
  return index

def index_all(df, index):
  for i in range(len(df)):
    entry = df.loc[i, :]
    index = index_it(entry, index)
  return index

def build_index(df, index):
    to_add = transform_df(df)
    index = index_all(df = to_add, index = index)
    return index

index = build_index(df = coventry_processed, index = {})



In [75]:
# Query Processor
irosyndex = index 
query = input("Search for:")
query = process_string(query)
retrieved = irosyndex[query]
coventry[coventry["ID"].isin(retrieved)]

,research title,publication link,abstract,author name,pureportal link,date,ID
13,Determinants of Informal Entrepreneurship in A...,https://pureportal.coventry.ac.uk/en/publicati...,This study investigates the determinants of in...,Hany Ahmed,https://pureportal.coventry.ac.uk/en/persons/h...,2022,14
15,Dissecting the effect of family business expos...,https://pureportal.coventry.ac.uk/en/publicati...,Purpose - Scholars have typically examined fam...,Abay Mulatu,https://pureportal.coventry.ac.uk/en/persons/a...,6 May 2022,16
32,Corporate Social Responsibility and Financial ...,https://pureportal.coventry.ac.uk/en/publicati...,This study investigates the determinants of in...,Thang Nguyen,https://pureportal.coventry.ac.uk/en/persons/t...,1 Apr 2022,33
34,"Diffusion theory, transnational antecedents an...",https://pureportal.coventry.ac.uk/en/publicati...,Purpose - Scholars have typically examined fam...,Andrews Owusu,https://pureportal.coventry.ac.uk/en/persons/a...,Apr 2022,35
35,Dissecting the effect of family business expos...,https://pureportal.coventry.ac.uk/en/publicati...,Caring for the environment has become everyone...,Ruth Owusu-Mensah,https://pureportal.coventry.ac.uk/en/persons/r...,6 May 2022,36
...,...,...,...,...,...,...,...
616,Bohemianism and urban regeneration: A structur...,https://pureportal.coventry.ac.uk/en/publicati...,Until the recent global financial crisis (GFC)...,Dimitrios Serenis,https://pureportal.coventry.ac.uk/en/persons/d...,1 Aug 2015,617
619,Competitiveness and Determinants of Bank Profi...,https://pureportal.coventry.ac.uk/en/publicati...,"Purpose of the paper: In 2001, a new legislati...",Dimitris Serenis,https://pureportal.coventry.ac.uk/en/persons/d...,25 Jun 2015,620
624,Does Regulatory Environment affect Earnings Ma...,https://pureportal.coventry.ac.uk/en/publicati...,CSR is as an emerging concept in the Syrian bu...,Daniel Santamaria,https://pureportal.coventry.ac.uk/en/persons/d...,2015,625
628,Financial decisions and market performance: An...,https://pureportal.coventry.ac.uk/en/publicati...,Islamic banking is the fastest growing sector ...,Sarkar Kabir,https://pureportal.coventry.ac.uk/en/persons/s...,2015,629
